## lifelib interactions

Some interactions with the [lifelib BasicTerm_S](https://lifelib.io/libraries/basiclife/BasicTerm_S.html) actuarial cashflow model.

***Run cells in sequence (see Play button), and use sliders to interact***

In [ ]:
%pip install openpyxl

In [1]:
import pandas
from BasicTerm_SE_nomx import BasicTerm_SE

In [ ]:
%pip install ipywidgets

In [2]:
from ipywidgets import interact

## cashflow interaction (visual)

In [ ]:
%pip install altair

In [3]:
import altair as alt
import numpy as np

In [38]:
def interactive_cashflows(p=1):
    df = BasicTerm_SE.Projection[p].result_cf().reset_index().melt(id_vars=['index'], var_name='cashflow')
    df['value'] = np.where(df['cashflow'] != 'Premiums', df['value']*-1,df['value'])
    df['value'] = np.where(df['cashflow'] == 'Net Cashflow', df['value']*-1,df['value'])
    #df = df[df['cashflow'] != 'Net Cashflow']

    # Vega-Altair layered chart
    
    base = alt.Chart(df).properties(width=580, height=400)

    bars = base.mark_bar().transform_filter(filter="datum.cashflow != 'Net Cashflow'").encode(
            tooltip=['cashflow:N','value:Q','index:N'],
            x=alt.X('index:Q', scale=alt.Scale(domain=[0,200]), title="time"),
            y=alt.Y('value:Q', scale=alt.Scale(domain=[-5000*3,3000*3])),
            color='cashflow:N'
          )

    lines = base.mark_line(interpolate='step-after').transform_filter(filter="datum.cashflow == 'Net Cashflow'").encode(
            tooltip=['key:N','value:Q','index:N'],
            x=alt.X('index:Q', scale=alt.Scale(domain=[0,200]), title="time"),
            y=alt.Y('value:Q', scale=alt.Scale(domain=[-500,300])),
            color=alt.value('black')
          )

    return bars+lines

In [39]:
interact(interactive_cashflows, p=(1,10000))

interactive(children=(IntSlider(value=1, description='p', max=10000, min=1), Output()), _dom_classes=('widget-…

<function __main__.interactive_cashflows(p=1)>

# observations

- **pol 2259 at end date**

    - pol 4194 is just really small relatively
- pol 888 an example of fut. NB that runs
- ***pol 581 a NB pol that fails***

In [188]:
#BasicTerm_SE.Projection[1].premiums(-1)

# Model Points

In [183]:
interact(lambda p=1: BasicTerm_SE.Projection[0].model_point_table[p-1:p], p=(1,10000))

interactive(children=(IntSlider(value=1, description='p', max=10000, min=1), Output()), _dom_classes=('widget-…

<function __main__.<lambda>(p=1)>

## summary interaction

In [33]:
interact(lambda p: BasicTerm_SE.Projection[p].result_pv(), p=(1,10000))

interactive(children=(IntSlider(value=5000, description='p', max=10000, min=1), Output()), _dom_classes=('widg…

<function __main__.<lambda>(p)>

## cashflow interaction

In [34]:
interact(lambda p: BasicTerm_SE.Projection[p].result_cf(), p=(1,10000))

interactive(children=(IntSlider(value=5000, description='p', max=10000, min=1), Output()), _dom_classes=('widg…

<function __main__.<lambda>(p)>

.

.

.

.

.

.

.

.

.

## all-policies 🚧 WIP

In [43]:
[BasicTerm_SE.Projection[p].result_pv().assign(p=p) for p in [1,2,3]][2]

,Premiums,Claims,Expenses,Commissions,Net Cashflow,p
PV,1.104633e+06,802454.869486,36262.959961,0.0,265915.612121,3


In [44]:
from functools import reduce

In [45]:
#40s runtime: pandas.concat([BasicTerm_S.Projection[p].result_pv()[:1].assign(p=p) for p in range(1,10000) ])

In [169]:
dur0s = [(p, BasicTerm_SE.Projection[p].duration_mth(0)) for p in range(1,10000)]

In [170]:
filtered = map(lambda d: d[0], filter(lambda d: d[1]>0, dur0s))

In [171]:
policy_results = pandas.DataFrame([ ( p, BasicTerm_SE.Projection[p].duration_mth(0),BasicTerm_SE.Projection[p].age_at_entry(), BasicTerm_SE.Projection[p].sex(), BasicTerm_SE.Projection[p].policy_term(), BasicTerm_SE.Projection[p].sum_assured(), BasicTerm_SE.Projection[p].pv_net_cf(), ) for p in filtered ], columns=['p','duration_mth','age_at_entry','sex','policy_term','sum_assured','pv_net_cf'])

In [172]:
policy_results

,p,duration_mth,age_at_entry,sex,policy_term,sum_assured,pv_net_cf
0,1,1,47,M,10,622000,108625.005624
1,2,210,29,M,20,752000,-18339.113005
2,3,15,51,F,10,799000,265915.612121
3,4,125,32,F,20,422000,-18390.488283
4,5,55,28,M,15,605000,31967.360377
...,...,...,...,...,...,...,...
8218,9995,183,24,M,20,95000,-9693.309330
8219,9996,157,47,M,20,827000,-58097.748784
8220,9997,168,30,M,15,826000,-5569.052690
8221,9998,146,45,F,20,783000,-14810.533240


In [194]:
alt.data_transformers.disable_max_rows()

alt.Chart(policy_results).properties(width=480, height=200, title='BasicTerm_SE Profitability by Policy').mark_point().encode(
            x='sum_assured:Q',
            y='pv_net_cf:Q',
            #color='age_at_entry:Q',
            #color=alt.Color('age_at_entry:Q').scale(scheme='spectral'),
            color=alt.Color('duration_mth:Q').scale(scheme='spectral', reverse=False),
            shape='sex:N',
            row='policy_term:N',
            order='p'
          )

alt.Chart(...)